<a href="https://colab.research.google.com/github/Abhishek7081/AdversarialAttacks/blob/master/Genetic_on_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#support python 2&3
from __future__ import division, print_function, unicode_literals

#common imports
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from operator import add
import random
import logging
from tqdm import tqdm


# For pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error

#make notebook's output stable across runs
np.random.seed(42)


#Ignore useless warnings
import warnings
warnings.filterwarnings(action='ignore',message='^internal gelsd')
pd.options.mode.chained_assignment = None

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping


# Helper: Early stopping.
early_stopper = EarlyStopping(patience=5)


In [2]:
class Network():
    """Represent a network and let us operate on it.

    Currently only works for an MLP.
    """

    def __init__(self, nn_param_choices=None):
        """Initialize our network.

        Args:
            nn_param_choices (dict): Parameters for the network, includes:
                nb_neurons (list): [64, 128, 256]
                nb_layers (list): [1, 2, 3, 4]
                activation (list): ['relu', 'elu']
                optimizer (list): ['rmsprop', 'adam']
        """
        self.score = 0.
        self.nn_param_choices = nn_param_choices
        self.network = {}  # (dic): represents MLP network parameters

    def create_random(self):
        """Create a random network."""
        for key in self.nn_param_choices:
            self.network[key] = random.choice(self.nn_param_choices[key])

    def create_set(self, network):
        """Set network properties.

        Args:
            network (dict): The network parameters

        """
        self.network = network

    def train(self, dataset):
        """Train the network and record the score.

        Args:
            dataset (str): Name of dataset to use.

        """
        if self.score == 0.:
            self.score = train_and_score(self.network, dataset)

    def print_network(self):
        """Print out a network."""
        logging.info(self.network)
        logging.info("Network score: %.2f" % (self.score))

In [3]:

class Optimizer():
    """Class that implements genetic algorithm for MLP optimization."""

    def __init__(self, nn_param_choices, retain=0.4,
                 random_select=0.1, mutate_chance=0.2):
        """Create an optimizer.

        Args:
            nn_param_choices (dict): Possible network paremters
            retain (float): Percentage of population to retain after
                each generation
            random_select (float): Probability of a rejected network
                remaining in the population
            mutate_chance (float): Probability a network will be
                randomly mutated

        """
        self.mutate_chance = mutate_chance
        self.random_select = random_select
        self.retain = retain
        self.nn_param_choices = nn_param_choices

    def create_population(self, count):
        """Create a population of random networks.

        Args:
            count (int): Number of networks to generate, aka the
                size of the population

        Returns:
            (list): Population of network objects

        """
        pop = []
        for _ in range(0, count):
            # Create a random network.
            network = Network(self.nn_param_choices)
            network.create_random()

            # Add the network to our population.
            pop.append(network)

        return pop

    @staticmethod
    def fitness(network):
        """Return the score, which is our fitness function."""
        return network.score

    def grade(self, pop):
        """Find average fitness for a population.

        Args:
            pop (list): The population of networks

        Returns:
            (float): The average score of the population

        """
        summed = reduce(add, (self.fitness(network) for network in pop))
        return summed / float((len(pop)))

    def breed(self, mother, father):
        """Make two children as parts of their parents.

        Args:
            mother (dict): Network parameters
            father (dict): Network parameters

        Returns:
            (list): Two network objects

        """
        children = []
        for _ in range(2):

            child = {}

            # Loop through the parameters and pick params for the kid.
            for param in self.nn_param_choices:
                child[param] = random.choice(
                    [mother.network[param], father.network[param]]
                )

            # Now create a network object.
            network = Network(self.nn_param_choices)
            network.create_set(child)

            # Randomly mutate some of the children.
            if self.mutate_chance > random.random():
                network = self.mutate(network)

            children.append(network)

        return children

    def mutate(self, network):
        """Randomly mutate one part of the network.

        Args:
            network (dict): The network parameters to mutate

        Returns:
            (Network): A randomly mutated network object

        """
        # Choose a random key.
        mutation = random.choice(list(self.nn_param_choices.keys()))

        # Mutate one of the params.
        network.network[mutation] = random.choice(self.nn_param_choices[mutation])

        return network

    def evolve(self, pop):
        """Evolve a population of networks.

        Args:
            pop (list): A list of network parameters

        Returns:
            (list): The evolved population of networks

        """
        # Get scores for each network.
        graded = [(self.fitness(network), network) for network in pop]

        # Sort on the scores.
        graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=False)]

        # Get the number we want to keep for the next gen.
        retain_length = int(len(graded)*self.retain)

        # The parents are every network we want to keep.
        parents = graded[:retain_length]

        # For those we aren't keeping, randomly keep some anyway.
        for individual in graded[retain_length:]:
            if self.random_select > random.random():
                parents.append(individual)

        # Now find out how many spots we have left to fill.
        parents_length = len(parents)
        desired_length = len(pop) - parents_length
        children = []

        # Add children, which are bred from two remaining networks.
        while len(children) < desired_length:

            # Get a random mom and dad.
            male = random.randint(0, parents_length-1)
            female = random.randint(0, parents_length-1)

            # Assuming they aren't the same network...
            if male != female:
                male = parents[male]
                female = parents[female]

                # Breed them.
                babies = self.breed(male, female)

                # Add the children one at a time.
                for baby in babies:
                    # Don't grow larger than desired length.
                    if len(children) < desired_length:
                        children.append(baby)

        parents.extend(children)

        return parents


In [4]:
def get_fire():

    # function to load data
    fire = pd.read_csv("forestfires.csv")
    fire.head()

    months_to_remove = ['nov','jan','may']
    forest_fire = fire.drop(fire[fire.month.isin(months_to_remove)].index ,axis=0)

    #Grouping the the burnt area to get a better understanding
    forest_fire['area_cat'] = pd.cut(forest_fire['area'], bins=[0,5, 10, 50, 100, 1100], include_lowest=True, 
                                  labels=['0-5', '5-10', '10-50', '50-100', '>100'])

    # I chose to remove any area greater than 100. 
    #remove outliers
    forest_fire.drop(forest_fire[forest_fire.area>100].index,axis=0, inplace=True)

    forest_fire['temp_bins'] = pd.cut(forest_fire.temp, bins=[0, 15, 20, 25, 40], include_lowest=True, 
                                  labels=['0-15', '15-20', '20-25', '>25'])

    # As, I have assumed temp is a critical indicator, to have a golden test dataset I would prefer that both train and test has equally distributed temperature conditions. Stratified shuffle split will help us maintain the required strata compositions here.
    # try StratifiedKFold(n_splits=1, test_size=0.2, random_state=42) 
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 
    for train_index, test_index in split.split(forest_fire.values, forest_fire.temp_bins.values):
      st_train_set = forest_fire.iloc[train_index]
      st_test_set = forest_fire.iloc[test_index]

    #now lets drop the area_cat and temp_bins 
    for _ in (st_train_set, st_test_set):
      _.drop(['area_cat','temp_bins'], axis=1, inplace=True)

    #create a fresh copy of train to preprocess
    x_train = st_train_set.drop('area', axis=1)
    y_train = st_train_set.area.copy()
    categorical_attributes = list(x_train.select_dtypes(include=['object']).columns)
    numerical_attributes = list(x_train.select_dtypes(include=['float64', 'int64']).columns)

    num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                          ('std_scaler', StandardScaler()),
                          ])
    full_pipeline = ColumnTransformer([('num', num_pipeline, numerical_attributes),
                                    ('cat', OneHotEncoder(), categorical_attributes),
                                    ])
    x_train = full_pipeline.fit_transform(x_train)
    # #check the train data
    x_train = pd.DataFrame(x_train, columns= numerical_attributes[0:] + list(full_pipeline.named_transformers_.cat.categories_[0]) +
              list(full_pipeline.named_transformers_.cat.categories_[1]))
    x_train.pop("X");
    x_train.pop("Y");
    x_train.pop("apr");



    #create a fresh copy of test to preprocess
    x_test = st_test_set.drop('area', axis=1)
    y_test = st_test_set.area.copy()
    categorical_attributes = list(x_test.select_dtypes(include=['object']).columns)
    numerical_attributes = list(x_test.select_dtypes(include=['float64', 'int64']).columns)

    num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                          ('std_scaler', StandardScaler()),
                          ])
    full_pipeline = ColumnTransformer([('num', num_pipeline, numerical_attributes),
                                    ('cat', OneHotEncoder(), categorical_attributes),
                                    ])
    x_test = full_pipeline.fit_transform(x_test)
    # #check the test data
    x_test = pd.DataFrame(x_test, columns= numerical_attributes[0:] + list(full_pipeline.named_transformers_.cat.categories_[0]) +
              list(full_pipeline.named_transformers_.cat.categories_[1]))
    x_test.pop("X");
    x_test.pop("Y");

    return (x_train, y_train, x_test, y_test)



def compile_model(network, input_shape):

    nb_layers = network['nb_layers']
    nb_neurons = network['nb_neurons']
    activation = network['activation']
    optimizer = network['optimizer']

    model = Sequential()

    # Add each layer.
    for i in range(nb_layers):

        # Need input shape for first layer.
        if i == 0:
            model.add(Dense(nb_neurons, activation=activation, input_shape=input_shape))
        else:
            model.add(Dense(nb_neurons, activation=activation))

        model.add(Dropout(0.2)) 

    # Output layer.
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

    return model

def train_and_score(network, dataset):
    # Train the model, return test loss.

    x_train, y_train, x_test, y_test = get_fire()

    input_shape = (23,)
    model = compile_model(network, input_shape)

    model.fit(x_train, y_train, batch_size = 10, epochs = 10, verbose = 2, callbacks = [early_stopper], validation_data=(x_test, y_test)) # using early stoping, so no real limit

    y_pred= model.predict(x_test)

    model_mse = mean_squared_error(y_test, y_pred)
    model_rmse = np.sqrt(model_mse)

    return model_rmse  # 1 is accuracy. 0 is loss.

In [5]:
#  """Entry point to evolving the neural network. Start here."""
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import logging
# from optimizer import Optimizer
# from tqdm import tqdm

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    # datefmt='%m/%d/%Y %I:%M:%S %p',
    datefmt='%I:%M:%S %p',
    level=logging.DEBUG,
    filename='log.txt'
)

def train_networks(networks, dataset):
    """Train each network.

    Args:
        networks (list): Current population of networks
        dataset (str): Dataset to use for training/evaluating
    """
    pbar = tqdm(total=len(networks)) #tqdm is a progress bar library ( terminal progress bar, is used to specify the total number of expected iterations)
    for network in networks:
        network.train(dataset)
        pbar.update(1)
    pbar.close()

def get_average_score(networks):
    """Get the average score for a group of networks.

    Args:
        networks (list): List of networks

    Returns:
        float: The average score of a population of networks.

    """
    total_score = 0
    for network in networks:
        total_score += network.score

    return total_score / len(networks)

def generate(generations, population, nn_param_choices, dataset):
    """Generate a network with the genetic algorithm.

    Args:
        generations (int): Number of times to evolve the population
        population (int): Number of networks in each generation
        nn_param_choices (dict): Parameter choices for networks
        dataset (str): Dataset to use for training/evaluating

    """
    optimizer = Optimizer(nn_param_choices)
    networks = optimizer.create_population(population)

    # Evolve the generation. // either 20 times, or till the average score reaches desired score
    for i in range(generations):
        logging.info("***Doing generation %d of %d***" %
                     (i + 1, generations))

        # Train and get score for networks.
        train_networks(networks, dataset)

        # Get the average score for this generation.
        average_score = get_average_score(networks)

        # Print out the average score each generation.
        logging.info("Generation average: %.2f" % (average_score))
        logging.info('-'*80)

        # Evolve, except on the last iteration.
        if i != generations - 1:
            # Do the evolution.
            networks = optimizer.evolve(networks)

    # Sort our final population.
    networks = sorted(networks, key=lambda x: x.score, reverse=True)

    # Print out the top 5 networks.
    print_networks(networks[:5])

def print_networks(networks):
    """Print a list of networks.

    Args:
        networks (list): The population of networks

    """
    logging.info('-'*80)
    for network in networks:
        network.print_network()

def main():
    """Evolve a network."""
    generations = 10  # Number of times to evolve the population.
    population = 20  # Number of networks in each generation.
    dataset = 'forestfire'


    nn_param_choices = {
        'nb_neurons': [8, 16, 32, 64, 128, 256, 512],
        'nb_layers': [1, 2, 3, 4, 5],
        'activation': ['relu', 'elu', 'tanh', 'linear'],
        'optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax'],
    }
    
    logging.info("***Evolving %d generations with population %d***" %
                 (generations, population))

    generate(generations, population, nn_param_choices, dataset)




In [6]:
if __name__ == '__main__':
    main()

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/10
40/40 - 2s - loss: 229.6209 - mse: 229.6209 - val_loss: 227.5004 - val_mse: 227.5004
Epoch 2/10
40/40 - 0s - loss: 225.8376 - mse: 225.8376 - val_loss: 223.8950 - val_mse: 223.8950
Epoch 3/10
40/40 - 0s - loss: 222.8244 - mse: 222.8244 - val_loss: 218.8388 - val_mse: 218.8388
Epoch 4/10
40/40 - 0s - loss: 217.4977 - mse: 217.4977 - val_loss: 213.9323 - val_mse: 213.9323
Epoch 5/10
40/40 - 0s - loss: 214.5551 - mse: 214.5551 - val_loss: 208.9822 - val_mse: 208.9822
Epoch 6/10
40/40 - 0s - loss: 212.5761 - mse: 212.5761 - val_loss: 205.1170 - val_mse: 205.1170
Epoch 7/10
40/40 - 0s - loss: 207.9659 - mse: 207.9659 - val_loss: 201.9493 - val_mse: 201.9493
Epoch 8/10
40/40 - 0s - loss: 204.6868 - mse: 204.6868 - val_loss: 199.7366 - val_mse: 199.7366
Epoch 9/10
40/40 - 0s - loss: 203.3687 - mse: 203.3687 - val_loss: 197.4231 - val_mse: 197.4231
Epoch 10/10
40/40 - 0s - loss: 200.1378 - mse: 200.1378 - val_loss: 195.8310 - val_mse: 195.8310


  5%|▌         | 1/20 [00:04<01:16,  4.03s/it]

Epoch 1/10
40/40 - 1s - loss: 228.5571 - mse: 228.5571 - val_loss: 223.7388 - val_mse: 223.7388
Epoch 2/10
40/40 - 0s - loss: 224.5607 - mse: 224.5607 - val_loss: 219.5867 - val_mse: 219.5867
Epoch 3/10
40/40 - 0s - loss: 218.7886 - mse: 218.7886 - val_loss: 215.2539 - val_mse: 215.2539
Epoch 4/10
40/40 - 0s - loss: 214.1494 - mse: 214.1494 - val_loss: 210.3145 - val_mse: 210.3145
Epoch 5/10
40/40 - 0s - loss: 213.2863 - mse: 213.2863 - val_loss: 206.3602 - val_mse: 206.3602
Epoch 6/10
40/40 - 0s - loss: 210.8918 - mse: 210.8918 - val_loss: 202.0601 - val_mse: 202.0601
Epoch 7/10
40/40 - 0s - loss: 205.5724 - mse: 205.5724 - val_loss: 198.3972 - val_mse: 198.3972
Epoch 8/10
40/40 - 0s - loss: 203.1225 - mse: 203.1225 - val_loss: 195.9072 - val_mse: 195.9072
Epoch 9/10
40/40 - 0s - loss: 197.8581 - mse: 197.8581 - val_loss: 193.8111 - val_mse: 193.8111
Epoch 10/10
40/40 - 0s - loss: 194.7020 - mse: 194.7020 - val_loss: 191.9440 - val_mse: 191.9440


 10%|█         | 2/20 [00:07<01:10,  3.90s/it]

...

 90%|█████████ | 18/20 [00:46<00:09,  4.93s/it]

Epoch 1/10
40/40 - 1s - loss: 210.9317 - mse: 210.9317 - val_loss: 192.5778 - val_mse: 192.5778
Epoch 2/10
40/40 - 1s - loss: 194.0020 - mse: 194.0020 - val_loss: 187.4508 - val_mse: 187.4508
Epoch 3/10
40/40 - 1s - loss: 192.0661 - mse: 192.0661 - val_loss: 186.6944 - val_mse: 186.6944
Epoch 4/10
40/40 - 1s - loss: 189.3444 - mse: 189.3444 - val_loss: 185.7805 - val_mse: 185.7805
Epoch 5/10
40/40 - 1s - loss: 187.9284 - mse: 187.9284 - val_loss: 184.2729 - val_mse: 184.2729
Epoch 6/10
40/40 - 1s - loss: 186.2938 - mse: 186.2938 - val_loss: 183.9298 - val_mse: 183.9298
Epoch 7/10
40/40 - 1s - loss: 184.7134 - mse: 184.7134 - val_loss: 183.3368 - val_mse: 183.3368
Epoch 8/10
40/40 - 1s - loss: 184.6870 - mse: 184.6870 - val_loss: 183.1835 - val_mse: 183.1835
Epoch 9/10
40/40 - 1s - loss: 183.6463 - mse: 183.6463 - val_loss: 182.8451 - val_mse: 182.8451
Epoch 10/10
40/40 - 1s - loss: 181.3538 - mse: 181.3538 - val_loss: 182.7766 - val_mse: 182.7766


 95%|█████████▌| 19/20 [00:53<00:05,  5.80s/it]

Epoch 1/10
40/40 - 1s - loss: 213.7794 - mse: 213.7794 - val_loss: 197.3398 - val_mse: 197.3398
Epoch 2/10
40/40 - 1s - loss: 194.4586 - mse: 194.4586 - val_loss: 190.8976 - val_mse: 190.8976
Epoch 3/10
40/40 - 1s - loss: 192.0787 - mse: 192.0787 - val_loss: 189.6678 - val_mse: 189.6678
Epoch 4/10
40/40 - 1s - loss: 188.3087 - mse: 188.3087 - val_loss: 188.5193 - val_mse: 188.5193
Epoch 5/10
40/40 - 1s - loss: 185.6074 - mse: 185.6074 - val_loss: 188.2445 - val_mse: 188.2445
Epoch 6/10
40/40 - 1s - loss: 187.0811 - mse: 187.0811 - val_loss: 187.8506 - val_mse: 187.8506
Epoch 7/10
40/40 - 1s - loss: 186.6808 - mse: 186.6808 - val_loss: 187.2505 - val_mse: 187.2505
Epoch 8/10
40/40 - 1s - loss: 184.7225 - mse: 184.7225 - val_loss: 187.5186 - val_mse: 187.5186
Epoch 9/10
40/40 - 1s - loss: 182.1424 - mse: 182.1424 - val_loss: 187.6709 - val_mse: 187.6709
Epoch 10/10
40/40 - 1s - loss: 181.9192 - mse: 181.9192 - val_loss: 187.5496 - val_mse: 187.5496


100%|██████████| 20/20 [01:01<00:00,  3.08s/it]


In [ ]:
rec_gen=[]
for i in range(tol_max):
    rec_gen.append(rec(area_predictions_xgb,y_test,i))

In [9]:
# Define the max tolerance limit for REC curve x-axis
# For this problem this represents the absolute value of error in the prediction of the outcome i.e. area burned
tol_max=20
def rec(m,n,tol):
    if type(m)!='numpy.ndarray':
        m=np.array(m)
    if type(n)!='numpy.ndarray':
        n=np.array(n)
    l=m.size
    percent = 0
    for i in range(l):
        if np.abs(10**m[i]-10**n[i])<=tol:
            percent+=1
    return 100*(percent/l)

In [ ]:
plt.figure(figsize=(10,8))
plt.title("REC curve for various models\n",fontsize=20)
plt.xlabel("Absolute error (tolerance) in prediction ($ha$)",fontsize=15)
plt.ylabel("Percentage of correct prediction",fontsize=15)
plt.xticks([i for i in range(0,tol_max+1,1)],fontsize=13)
plt.ylim(-10,100)
plt.xlim(-2,tol_max)
plt.yticks([i*20 for i in range(6)],fontsize=18)
plt.grid(True)
# plt.plot(range(tol_max),rec_XGB ,'--',lw=3)
plt.plot(range(tol_max),rec_gen,'k-',lw=3)
plt.legend(['SVR','XGB'],fontsize=13)